In [54]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def preprocess_shakespeare(text, seq_length=40, stride=3):
    chars = sorted(list(set(text))) 
    char_to_ix = {ch: i for i, ch in enumerate(chars)}
    ix_to_char = {i: ch for i, ch in enumerate(chars)}

    data_indices = [char_to_ix[ch] for ch in text]

    sequences = []
    targets = []
    for start_idx in range(0, len(data_indices) - seq_length, stride):
        seq = data_indices[start_idx : start_idx + seq_length]
        target = data_indices[start_idx + 1 : start_idx + seq_length + 1]
        sequences.append(seq)
        targets.append(target)

    return sequences, targets, char_to_ix, ix_to_char

with open(r"C:\Users\yosub\CSE-151B\HW3_Public\HW3_Public\poem_data\shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

seq_length = 40
stride = 3
sequences, targets, char_to_ix, ix_to_char = preprocess_shakespeare(text, seq_length, stride)

print(f"Number of sequences: {len(sequences)}")
print(f"Vocab size: {len(char_to_ix)}")
print(f"Example sequence (chars): {''.join([ix_to_char[ix] for ix in sequences[0]])}")
print(f"Example target (chars): {''.join([ix_to_char[ix] for ix in targets[0]])}")


Number of sequences: 32663
Vocab size: 71
Example sequence (chars):                    1
From fairest creatu
Example target (chars):                   1
From fairest creatur


In [ ]:
import torch
import torch.nn as nn

class CharLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_size=128, num_layers=1):
        super(CharLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)  
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_size,
                            num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, vocab_size)              

    def forward(self, x, hidden=None):
        # x shape: (seq_len, batch_size)
        embedded = self.embedding(x)                              # (seq_len, batch_size, embedding_dim)
        output, hidden = self.lstm(embedded, hidden)             # output: (seq_len, batch_size, hidden_size)
        logits = self.fc(output)                                  # (seq_len, batch_size, vocab_size)
        return logits, hidden

embedding_dim = 128
hidden_size = 128
num_layers = 1

vocab_size = len(char_to_ix)

model = CharLSTM(vocab_size, embedding_dim, hidden_size, num_layers).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

print(model)


CharLSTM(
  (embedding): Embedding(71, 128)
  (lstm): LSTM(128, 128)
  (fc): Linear(in_features=128, out_features=71, bias=True)
)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

inputs = torch.tensor(sequences)    # (num_samples, seq_length)
labels = torch.tensor(targets)      # (num_samples, seq_length)

dataset = TensorDataset(inputs, labels)
batch_size = 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch_inputs, batch_labels in dataloader:
        batch_inputs = batch_inputs.transpose(0,1).to(device)   # (seq_len, batch)
        batch_labels = batch_labels.transpose(0,1).to(device)

        batch_size = batch_inputs.size(1)

        hidden = None

        optimizer.zero_grad()
        output, hidden = model(batch_inputs, hidden)
        hidden = (hidden[0].detach(), hidden[1].detach())

        output = output.reshape(-1, vocab_size)
        batch_labels = batch_labels.reshape(-1)

        loss = loss_fn(output, batch_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} \t Loss: {avg_loss:.4f}")



Epoch 1/20 	 Loss: 1.8421
Epoch 2/20 	 Loss: 1.5045
Epoch 3/20 	 Loss: 1.4066
Epoch 4/20 	 Loss: 1.3410
Epoch 5/20 	 Loss: 1.2892
Epoch 6/20 	 Loss: 1.2461
Epoch 7/20 	 Loss: 1.2087
Epoch 8/20 	 Loss: 1.1760
Epoch 9/20 	 Loss: 1.1471
Epoch 10/20 	 Loss: 1.1219
Epoch 11/20 	 Loss: 1.1007
Epoch 12/20 	 Loss: 1.0818
Epoch 13/20 	 Loss: 1.0646
Epoch 14/20 	 Loss: 1.0494
Epoch 15/20 	 Loss: 1.0363
Epoch 16/20 	 Loss: 1.0248
Epoch 17/20 	 Loss: 1.0138
Epoch 18/20 	 Loss: 1.0047
Epoch 19/20 	 Loss: 0.9951
Epoch 20/20 	 Loss: 0.9876


In [ ]:
def generate_text(model, seed_text, char_to_ix, ix_to_char, length=200, temperature=1.0):
    model.eval()
    generated = seed_text

    input_seq = [char_to_ix[ch] for ch in seed_text]
    input_tensor = torch.tensor(input_seq).unsqueeze(1).to(device)  # (seq_len, 1)

    hidden = None
    with torch.no_grad():
        output, hidden = model(input_tensor, hidden)

        for _ in range(length):
            last_logits = output[-1].squeeze()  # (vocab_size,)

            scaled_logits = last_logits / temperature

            probs = torch.softmax(scaled_logits, dim=-1)

            next_char_idx = torch.multinomial(probs, num_samples=1).item()

            generated += ix_to_char[next_char_idx]

            input_tensor = torch.tensor([[next_char_idx]]).to(device)
            output, hidden = model(input_tensor, hidden)

    return generated


In [60]:
seed = "shall i compare thee to a summers day?\n"
print(generate_text(model, seed, char_to_ix, ix_to_char, length=200, temperature=0.75))


shall i compare thee to a summers day?
It fears the objechsing thy heart the world and thy self dost before,
  Theirs argood made, all you shalt spirit did?
Shall will transport by lambry?
Is it not to pry,
How with thy sum thy show alone.


In [61]:
seed = "shall i compare thee to a summers day?\n"
print(generate_text(model, seed, char_to_ix, ix_to_char, length=200, temperature=1.5))


shall i compare thee to a summers day?
That assamed, or crow, oun pull you behom,
  And they memn hides thou, nothing no frowness
Skencr,
On subt, yet give he be doth my sit,
Duteous) is flown abument dirept thou servyouths hands of thines


In [62]:
seed = "shall i compare thee to a summers day?\n"
print(generate_text(model, seed, char_to_ix, ix_to_char, length=200, temperature=0.25))


shall i compare thee to a summers day?
If thou gav'st and look of earth to be thy complexion did cruel.


                   138
When my love that which it self-substance stol'n false of thy beauty shall I not the store that she so faults 
